In [0]:
from pyspark.sql.functions import *

In [0]:
RAW_STATIONS = "/Volumes/rail_lakehouse/bronze_rail/raw_files/stations"
CHECKPOINT_STATIONS = "/Volumes/rail_lakehouse/bronze_rail/checkpoints/stations" 

print(f"📂 Raw stations: {RAW_STATIONS}")
print(f"💾 Checkpoint: {CHECKPOINT_STATIONS}")

In [0]:
df_bronze_stations = (
    spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.schemaLocation", f"{CHECKPOINT_STATIONS}/schema")
    .load(RAW_STATIONS)
)


In [0]:
df_bronze_source = ( 
    df_bronze_stations
    .select(
        col("stationShortCode").alias("station_code"),
        col("stationName").alias("station_name"),
        col("type"),
        col("stationUICCode"),
        current_timestamp().alias("ingest_ts")
    )
)

In [0]:
query_bronze_stations = (
    df_bronze_source.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{CHECKPOINT_STATIONS}/bronze")
    .trigger(availableNow=True)
    .table("rail_lakehouse.bronze_rail.bronze_stations")
)



In [0]:
query_bronze_stations.awaitTermination()

In [0]:
spark.sql("SELECT COUNT(*) FROM rail_lakehouse.bronze_rail.bronze_stations").show()
spark.sql("DESCRIBE rail_lakehouse.bronze_rail.bronze_stations").show()

In [0]:
%sql
SELECT * 
FROM rail_lakehouse.bronze_rail.bronze_stations 
LIMIT 5;